In [2]:
import pandas as pd
import json

In [11]:
with open('VietnameLocations.json', mode='r', encoding='utf-8') as file:
    data = json.load(file)
type(data)

list

In [29]:
# data[x] -> 1 tỉnh
# data[x][0] -> mã tỉnh (không phải từ 1 - 63) -> muốn là từ 1 đến 63
# data[x][1] -> tên tỉnh (tiếng Việt)
# data[x][2] -> tiền tố (tỉnh/thành phố)
# data[x][4] -> dữ liệu các huyện/quận của tỉnh/thành phố
# data[x][4][y] -> dữ liệu của huyện thứ y (trong tỉnh thứ x)
# data[x][4][y][0] -> mã huyện/quận
# data[x][4][y][1] -> tên huyện/quận (tiếng Việt)
# data[x][4][y][2] -> tiền tố (huyện/quận/thị trấn)
# data[x][4][y][4] -> dữ liệu các xã/phường của quận/huyện
# data[x][4][y][4][z] tên huyện thứ z của quận huyện thứ y của tỉnh thứ x
# data[x][4][y][4][z][0] -> mã xã/phường
# data[x][4][y][4][z][1] -> tên xã/phường tiếng Việt
# data[x][4][y][4][z][2] -> tiền tố
data[0][4][0][4]

[['30337', 'An Phú', 'Thị trấn', 'An Phu'],
 ['30373', 'Đa Phước', 'Thị trấn', 'Da Phuoc'],
 ['30340', 'Khánh An', 'Xã', 'Khanh An'],
 ['30343', 'Khánh Bình', 'Xã', 'Khanh Binh'],
 ['30341', 'Long Bình', 'Thị Trấn', 'Long Binh'],
 ['30349', 'Nhơn Hội', 'Xã', 'Nhon Hoi'],
 ['30355', 'Phú Hội', 'Xã', 'Phu Hoi'],
 ['30352', 'Phú Hữu', 'Xã', 'Phu Huu'],
 ['30358', 'Phước Hưng', 'Xã', 'Phuoc Hung'],
 ['30346', 'Quốc Thái', 'Xã', 'Quoc Thai'],
 ['30364', 'Vĩnh Hậu', 'Xã', 'Vinh Hau'],
 ['30370', 'Vĩnh Hội Đông', 'Xã', 'Vinh Hoi Dong'],
 ['30361', 'Vĩnh Lộc', 'Xã', 'Vinh Loc'],
 ['30367', 'Vĩnh Trường', 'Xã', 'Vinh Truong']]

In [32]:
# Initialize the final dictionary
locations_dict = {}

# Process the data
for province in data:
    province_name = province[1]  # Province name
    
    # Initialize the dictionary for districts in the current province
    districts_dict = {}
    
    # Iterate through districts in the province
    for district in province[4]:
        district_name = district[2] + ' ' + district[1]  # District name
        
        # Collect ward names for this district
        wards = [ward[2] + ' ' + ward[1] for ward in district[4]]  # List of ward names
        
        # Assign the district and its wards to the districts dictionary
        districts_dict[district_name] = wards
    
    # Assign the province and its districts dictionary to the final dictionary
    locations_dict[province_name] = districts_dict

# Now `locations_dict` contains the structured data

In [35]:
locations_dict['Hồ Chí Minh']['Thành phố Thủ Đức']

['Phường An Khánh',
 'Phường An Lợi Đông',
 'Phường An Phú',
 'Phường Bình Chiểu',
 'Phường Bình Thọ',
 'Phường Bình Trưng Đông',
 'Phường Bình Trưng Tây',
 'Phường Cát Lái',
 'Phường Hiệp Bình Chánh',
 'Phường Hiệp Bình Phước',
 'Phường Hiệp Phú',
 'Phường Linh Chiểu',
 'Phường Linh Đông',
 'Phường Linh Tây',
 'Phường Linh Trung',
 'Phường Linh Xuân',
 'Phường Long Bình',
 'Phường Long Phước',
 'Phường Long Thạnh Mỹ',
 'Phường Long Trường',
 'Phường Phú Hữu',
 'Phường Phước Bình',
 'Phường Phước Long A',
 'Phường Phước Long B',
 'Phường Tam Bình',
 'Phường Tam Phú',
 'Phường Tân Phú',
 'Phường Tăng Nhơn Phú A',
 'Phường Tăng Nhơn Phú B',
 'Phường Thạnh Mỹ Lợi',
 'Phường Thảo Điền',
 'Phường Thủ Thiêm',
 'Phường Trường Thạnh',
 'Phường Trường Thọ']

In [54]:
# List to store SQL queries
sql_queries = []

# Track province and district IDs manually for simplicity
province_id = 1
district_id = 1
ward_id = 1

# SQL query template for provinces, districts, and wards
province_query_template = 'INSERT INTO store_province(id, name) VALUES ({}, "{}");'
district_query_template = 'INSERT INTO store_district(id, name, province_id) VALUES ({}, "{}", {});'
ward_query_template = 'INSERT INTO store_ward(id, name, district_id) VALUES ({}, "{}", {});'

In [55]:
# Iterate through provinces, districts, and wards to create SQL queries
for province_name, districts in locations_dict.items():
    # Insert province
    sql_queries.append(province_query_template.format(province_id, province_name))
    
    # Iterate through districts in the current province
    for district_name, wards in districts.items():
        # Insert district, linked to the current province_id
        sql_queries.append(district_query_template.format(district_id, district_name, province_id))
        
        # Iterate through wards in the current district
        for ward_name in wards:
            # Insert ward, linked to the current district_id
            sql_queries.append(ward_query_template.format(ward_id, ward_name, district_id))
            ward_id += 1  # Increment ward_id for the next ward
        
        district_id += 1  # Increment district_id for the next district
    
    province_id += 1  # Increment province_id for the next province

# Join all queries into one string (or write them to a file)
sql_script = "\n".join(sql_queries)

In [56]:
with open('sql_script.txt', mode='w', encoding='utf-8') as file:
    for query in sql_queries:
        file.write(query + '\n')